# Policy Gradient with Discrete Action Space
**Author:** Marc-Philip Ecker

## Imports

In [ ]:
import torch
import torch.nn as nn
from torch.distributions.categorical import Categorical
from torch.optim import Adam
import numpy as np
import gym
import matplotlib.pyplot as plt

## Environment
In this exercise we develop an RL-agent for the gymnasium cartpole environemen (see https://gymnasium.farama.org/environments/classic_control/cart_pole/).

In [ ]:
env_name = "CartPole-v1"
env = gym.make(env_name)

## Define the Policy
In this section we defining the policy as a neural network. The policy $\pi_{\boldsymbol{\theta}}(\mathbf{u}|\mathbf{x})$ defines the probabilities of taking an action $\mathbf{u}\in\mathcal{U}$ given that the environment is in a certain state $\mathbf{x}\in\mathcal{X}$. The action space for the gymnasium CartPole-v1 environment is discrete, i.e. $\mathcal{U}=\{\mathbf{u}_1,\mathbf{u}_2\}$. We create a neural network that takes as input the current state $\mathbf{x}\in\mathcal{X}$ and returns the unnormalized log probabilities $\log\pi_{\boldsymbol{\theta}}(\mathbf{u}|\mathbf{x})$ (logits). Since, $\mathcal{X}\subseteq\mathbb{R}^n$, the layer requires $n$ input units to handle the state as input. We use one hidden layer with 32 units and $m=2$ outputs, which represent the log probabilities.

Hence, the network is defined by the parameters $\boldsymbol{\theta}=\{\mathbf{W}_1,\mathbf{b}_1,\mathbf{W}_2,\mathbf{b}_2\}$, where $\mathbf W_1\in\mathbb{R}^{n\times 32}$, $\mathbf{W}_2\in\mathbb{R}^{32\times m}$ are the weighting matrices of the linear units and $\mathbf{b}_1\in\mathbb{R}^{32}$, $\mathbf{b}_2\in\mathbb{R}^{m}$ are the bias vectors.

In [ ]:
# Get dimensions of state space and action space from environment
n = env.observation_space.shape[0];
m = env.action_space.n;

# Define layers sizes
layer_sizes = [n, 32, 32, m]

# Set layers
layers = []
for i in range(len(layer_sizes)-1):
  act = nn.Tanh if i < len(layer_sizes) - 2 else nn.Identity
  layers += [nn.Linear(layer_sizes[i], layer_sizes[i+1]), act()]

# Define policy
policy_network = nn.Sequential(*layers)

# Reset network parameters
def reset_policy():
  for layer in policy_network.children():
    if hasattr(layer, 'reset_parameters'):
        layer.reset_parameters()

We define a distribution that takes the outputs of the neural network as input. In this case we use the Categorical distribution (https://pytorch.org/docs/stable/distributions.html#categorical) which either takes as input the log probabilities or the probabilities. Since our network returns the log probabilities, we use these as input. The probabilities are then computed using the Softmax function
\begin{align}
  \pi_{\boldsymbol{\theta}}(\mathbf{u}_i|\mathbf{x})=\frac{\exp(\log\pi_{\boldsymbol{\theta}}(\mathbf{u}_i|\mathbf{x}))}{\sum_{i=1}^2\exp(\log\pi_{\boldsymbol{\theta}}(\mathbf{u}_i|\mathbf{x}))}\ .
\end{align}
We can use this distribution to generate samples from our policy.

In [ ]:
def get_policy(x):
  logits = policy_network(x)
  return Categorical(logits=logits)

def sample_action(x):
  return get_policy(x).sample().item()

In [ ]:
def visualize_policy():
  env_local = gym.make(env_name, render_mode="human")
  x = env_local.reset()[0]
  while True:
    env_local.render()
    u = sample_action(torch.as_tensor(x, dtype=torch.float32))
    x, _, done, _, _ = env_local.step(u)

    if done:
      x = env_local.reset()[0]
      break
    
  env_local.close()

visualize_policy()

## Policy Gradient

In RL, the goal is to maximize the sum of rewards
\begin{align}
  \max_{\boldsymbol\theta}J(\boldsymbol{\theta})=\mathbb{E}_{\boldsymbol{\tau}\sim p_{\boldsymbol{\theta}}}\Big\{\sum_{k=0}^{N-1}r(\mathbf{x}_k,\mathbf{u}_k)\Big\}\ .
\end{align}
Given a set of i.i.d. sample trajectories $\boldsymbol{\tau}^{[i]}=\{\mathbf{x}_0^{[i]},\mathbf{u}_0^{[i]}, \dots,\mathbf{x}_{N-1}^{[i]},\mathbf{u}_{N-1}^{[i]}\}$, $i=1,\dots,N_{\mathrm{sample}}$, an unbiased estimator of the expectation can be computed
\begin{align}
  J(\boldsymbol{\theta})\approx\frac{1}{N_{\mathrm{sample}}}\sum_{i=1}^{N_{\mathrm{sample}}}\sum_{k=0}^{N-1}r(\mathbf{x}_k,\mathbf{u}_k)\ .
\end{align}
The goal of policy gradient is to adapt parameters of the policy s.t. the expected return increases. This is achieved using gradient ascent $\boldsymbol{\theta}\leftarrow\boldsymbol{\theta}+\alpha\nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta})$. The policy gradient is given by
\begin{align}
  \nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta})=\mathbb{E}_{\boldsymbol{\tau}\sim p_{\boldsymbol{\theta}}}\Big\{\sum_{k=0}^{N-1}\nabla_{\boldsymbol{\theta}}\log\pi_{\boldsymbol{\theta}}(\mathbf{u}_k|\mathbf{x}_k)\sum_{k=0}^{N-1}r(\mathbf{x}_k,\mathbf{u}_k)\Big\}
\end{align}
and it can be estimated using
\begin{align}
  \nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta})\approx\frac{1}{N_{\mathrm{sample}}}\sum_{i=1}^{N_{\mathrm{sample}}}\Big\{\sum_{k=0}^{N-1}\nabla_{\boldsymbol{\theta}}\log\pi_{\boldsymbol{\theta}}(\mathbf{u}_k^{[i]}|\mathbf{x}_k^{[i]})\sum_{k=0}^{N-1}r(\mathbf{x}_k^{[i]},\mathbf{u}_k^{[i]})\Big\}\ .
\end{align}

Using the loss function
\begin{align}
  L(\boldsymbol{\theta})=-\frac{1}{N_{\mathrm{sample}}}\sum_{i=1}^{N_{\mathrm{sample}}}\Big\{\sum_{k=0}^{N-1}\log\pi_{\boldsymbol{\theta}}(\mathbf{u}_k^{[i]}|\mathbf{x}_k^{[i]})\sum_{k=0}^{N-1}r(\mathbf{x}_k^{[i]},\mathbf{u}_k^{[i]})\Big\}
\end{align}
allows to use the autodiff functionality of PyTorch.


In [ ]:
def get_loss(x, u, R):
  log_pi = get_policy(x).log_prob(u)
  return -(log_pi * R).mean()

The main algorthm works as follows:

For each epoch:

1.   Generate sample trajectories $\boldsymbol{\tau}^{[i]}$, $i=1,\dots,N_{\mathrm{sample}}$ and the corresponding returns $R(\boldsymbol{\tau})=\sum_{k=0}^{N-1}r(\mathbf{x}_k^{[i]},\mathbf{u}_k^{[i]})$
2.   Compute the policy gradient $\nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta})$. This is done by PyTorch backprop.
3.   Perform gradient ascent step $\boldsymbol{\theta}\leftarrow\boldsymbol{\theta}+\alpha\nabla_{\boldsymbol{\theta}}J(\boldsymbol{\theta})$ (PyTorch optimizer.step())



In [ ]:
def train_one_epoch(optimizer, batch_size):
  x_batch = []
  u_batch = []
  R_batch = []
  w_batch = []
  N_batch = []

  x = env.reset()[0]
  rewards = []

  # Sample trajectories
  while True:
    # Sample action
    u = sample_action(torch.as_tensor(x, dtype=torch.float32))
    # Store state action pair
    x_batch.append(x)
    u_batch.append(u)
    # Apply action to environment
    x, r, done, _, _ = env.step(u)
    # Store reward
    rewards.append(r)

    if done:
      # Compute return and episode length
      R = sum(rewards)
      N = len(rewards)
      # Store return and episode length
      R_batch.append(R)
      N_batch.append(N)
      # Compute weights for loss. There are N terms that are weighted
      # by the return
      w_batch += [R] * N

      x = env.reset()[0]
      done = False
      rewards = []

      if len(x_batch) > batch_size:
        break

  optimizer.zero_grad()
  # Define the loss for the current batch
  batch_loss = get_loss(torch.as_tensor(x_batch, dtype=torch.float32),
                        torch.as_tensor(u_batch, dtype=torch.float32),
                        torch.as_tensor(w_batch, dtype=torch.float32))
  # Compute gradient
  batch_loss.backward()
  # Make a gradient ascent step
  optimizer.step()

  return batch_loss, R_batch, N_batch

In [ ]:
def train(epochs=50, batch_size=5000, learning_rate=1e-2):
  optimizer = Adam(policy_network.parameters(), learning_rate)

  returns = []
  for i in range(epochs):
    batch_loss, R_batch, N_batch = train_one_epoch(optimizer, batch_size)
    returns.append(np.mean(R_batch))
    print('epoch: %3d \t loss: %.3f \t return: %.3f \t episode length: %.3f'%
                (i, batch_loss, np.mean(R_batch), np.mean(N_batch)))
  return returns

In [ ]:
reset_policy()
returns = train()
visualize_policy()

## Causality

In [ ]:
def get_reward_to_go(rewards):
  N = len(rewards)
  r_to_go = [0] * N
  for i in reversed(range(N)):
    r_to_go[i] = rewards[i] + (r_to_go[i+1] if i < N-1 else 0)

  return r_to_go

def train_one_epoch(optimizer, batch_size):
  x_batch = []
  u_batch = []
  R_batch = []
  w_batch = []
  N_batch = []

  x = env.reset()[0]
  rewards = []

  # Sample trajectories
  while True:
    # Sample action
    u = sample_action(torch.as_tensor(x, dtype=torch.float32))
    # Store state action pair
    x_batch.append(x)
    u_batch.append(u)
    # Apply action to environment
    x, r, done, _, _ = env.step(u)
    # Store reward
    rewards.append(r)

    if done:
      # Compute return and episode length
      R = sum(rewards)
      N = len(rewards)
      # Store return and episode length
      R_batch.append(R)
      N_batch.append(N)
      # Compute weights for loss. There are N terms that are weighted
      # by the return
      w_batch += get_reward_to_go(rewards)

      x = env.reset()[0]
      done = False
      rewards = []

      if len(x_batch) > batch_size:
        break

  optimizer.zero_grad()
  # Define the loss for the current batch
  batch_loss = get_loss(torch.as_tensor(x_batch, dtype=torch.float32),
                        torch.as_tensor(u_batch, dtype=torch.float32),
                        torch.as_tensor(w_batch, dtype=torch.float32))
  # Compute gradient
  batch_loss.backward()
  # Make a gradient ascent step
  optimizer.step()

  return batch_loss, R_batch, N_batch

In [ ]:
reset_policy()
returns_causality = train()
visualize_policy()

## Baseline

In [ ]:
def train_one_epoch(optimizer, batch_size):
  x_batch = []
  u_batch = []
  R_batch = []
  w_batch = []
  N_batch = []

  x = env.reset()[0]
  rewards = []

  # Sample trajectories
  while True:
    # Sample action
    u = sample_action(torch.as_tensor(x, dtype=torch.float32))
    # Store state action pair
    x_batch.append(x)
    u_batch.append(u)
    # Apply action to environment
    x, r, done, _, _ = env.step(u)
    # Store reward
    rewards.append(r)

    if done:
      # Compute return and episode length
      R = sum(rewards)
      N = len(rewards)
      # Store return and episode length
      R_batch.append(R)
      N_batch.append(N)
      # Compute weights for loss. There are N terms that are weighted
      # by the return
      w_batch += [R] * N

      x = env.reset()[0]
      done = False
      rewards = []

      if len(x_batch) > batch_size:
        # Baseline
        R_mean = np.mean(R_batch)
        for i in range(len(w_batch)):
          w_batch[i] -= R_mean
        break

  optimizer.zero_grad()
  # Define the loss for the current batch
  batch_loss = get_loss(torch.as_tensor(x_batch, dtype=torch.float32),
                        torch.as_tensor(u_batch, dtype=torch.float32),
                        torch.as_tensor(w_batch, dtype=torch.float32))
  # Compute gradient
  batch_loss.backward()
  # Make a gradient ascent step
  optimizer.step()

  return batch_loss, R_batch, N_batch

In [ ]:
reset_policy()
returns_baseline = train()
visualize_policy()

## Compare results

In [ ]:
plt.plot(returns)
plt.plot(returns_causality)
plt.plot(returns_baseline)
plt.show()